# Análisis de diversos concierto y eventos culturales que se realizan en el Perú

En este proyecto de análisis, nos proponemos explorar y comparar detalladamente los distintos conciertos que se llevan a cabo en nuestro país. La investigación se centrará en revelar aspectos clave de cada evento, incluyendo el tipo de acontecimiento planificado, la destacada participación de artistas o grupos musicales, así como el medio o lugar específico donde se llevará a cabo cada concierto. A través de este análisis, buscamos proporcionar una visión integral del panorama musical en nuestro país, destacando no solo la diversidad de eventos, sino también brindando información valiosa para aquellos interesados en participar o conocer más sobre la escena musical local. La recopilación y comparación de estos datos permitirán identificar tendencias, preferencias y posiblemente contribuir a la planificación de futuros eventos en base a la demanda y éxito previos.
Para ello vamos a utilizar los siguientes paquetes:

-*Request*: es una biblioteca de Python que facilita el envío de solicitudes HTTP. Permite realizar peticiones a servidores web y gestionar las respuestas de manera sencilla. Con este paquete, puedes realizar operaciones como GET, POST, PUT, DELETE, entre otras, de manera fácil y eficiente. Además, proporciona funcionalidades para el manejo de sesiones, cookies y encabezados HTTP. Es ampliamente utilizado para interactuar con APIs web y realizar solicitudes HTTP en proyectos Python.

-*bs4*: Beautiful Soup proporciona herramientas para extraer información de documentos HTML o XML, facilitando el web scraping y la manipulación de datos estructurados en la web.

-*Pandas*: es una biblioteca de análisis de datos que proporciona estructuras de datos flexibles y herramientas de manipulación de datos para facilitar la limpieza, exploración y análisis de datos.

-*Re*: Las expresiones regulares (regex) son patrones de búsqueda y manipulación de texto que permiten realizar operaciones avanzadas en cadenas de caracteres.


Y de las páginas en las cuales analizaremos estos eventos son:

-Teleticket: es una plataforma de venta de boletos y gestión de eventos en línea. Proporciona servicios para la compra de entradas para diversos eventos, como conciertos, espectáculos y actividades culturales en países específicos, con un enfoque regional.

-Kandavu: es una plataforma de gestión de eventos y venta de boletos que ofrece soluciones para la organización y promoción de eventos en línea. Facilita la venta de entradas, el control de acceso y la administración de eventos, brindando a organizadores y asistentes herramientas para una experiencia exitosa.

-Ticketmaster: es una de las mayores empresas a nivel mundial en la venta de boletos para eventos en vivo, incluyendo conciertos, deportes, teatro y otros espectáculos. Proporciona una plataforma en línea para la compra segura de entradas, así como servicios de gestión de eventos y distribución de boletos para organizadores. 

In [1]:
# Paquetes
import requests
import bs4 as bs 
import pandas as pd 
import re

In [2]:
#Kandavu

url = "https://www.kandavu.com/"
response = requests.get(url)
content = response.text
soup = bs.BeautifulSoup(content, 'lxml')

names = soup.find_all('h4', class_ = 'g1_title')
nombres = [h4.text.strip() for h4 in names]
nombres = nombres[8:11]

date = soup.find_all('time')
fecha = [time.text.strip() for time in date]
fecha

data = {'Nombre':nombres, 'Fecha':fecha}
df = pd.DataFrame(data)

# Centrar el encabezado y el texto

estilos = [
    dict(selector="th", props=[("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")])
]

# Aplicar el estilo al DataFrame
df_estilizado = df.style.set_table_styles(estilos)

df


,Nombre,Fecha
0,LIMA MUSIC FEST LATIN PARTY EDITION,29 de February de 2024
1,LAURA PAUSINI,06 de March de 2024
2,MEGADETH,06 de April de 2024


In [3]:
# Ticket Master

url = "https://www.ticketmaster.pe/"
response = requests.get(url)
content = response.text
soup = bs.BeautifulSoup(content, 'lxml')

names = soup.find_all('h3', class_ = 'thumb-title')
nombres = [h3.text.strip() for h3 in names]
nombres = list(set(nombres)) # Para eliminar los valores que se repiten en la lista

date_hour = soup.find_all('span', class_ = 'event_description')
descr_fecha_hora= [span.text.strip() for span in date_hour]
descr_fecha_hora = list(set(descr_fecha_hora))

# Expresión regular para extraer fechas
regex_fecha = re.compile(r' \b(?:\d{1,2}\s+(?:de\s+)?(?:Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)(?:\s+del)?\s+(?:\d{2,4})(?:\s*-\s*\d{1,2}\s+(?:[ap]\.m\.|\d{1,2}:\d{2}\s*(?:[ap]\.m\.)?)?)?|\d{1,2}\s*-\s*\d{1,2}\s+(?:de\s+)?(?:Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)(?:\s+del)?\s+(?:\d{2,4})|(?:\d{1,2}\s*(?:[ap]\.m\.|\d{1,2}:\d{2}\s*(?:[ap]\.m\.)?)?)\s*-\s*(?:Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)(?:\s+del)?\s+(?:\d{2,4}))\b ', re.IGNORECASE)

# Separar fechas y descripciones
fechas = [match.group() for event in descr_fecha_hora if (match := regex_fecha.search(event))]
descripciones = [event for event in descr_fecha_hora if not regex_fecha.search(event)]

# Imprimir resultados

regex_dia_semana = re.compile(r'\b(Lunes|Martes|Miércoles|Jueves|Viernes|Sábado|Domingo)\b', re.IGNORECASE)

# Crear lista para almacenar días de la semana
dias_semana = []

# Filtrar días de la semana
for evento in descripciones:
    if regex_dia_semana.search(evento):
        dias_semana.append(evento)

data = {'Nombre':nombres}
df = pd.DataFrame(data)
df['Fecha'] = None
df['Descripcion'] = None

new_value = descripciones[8]
df.at[0, 'Descripcion'] = new_value

new_value1 = descripciones[1]
df.at[1, 'Fecha'] = new_value1

new_value2 = descripciones[7]
df.at[2, 'Descripcion'] = new_value2

new_value3 = fechas[1]
df.at[3, 'Fecha'] = new_value3

new_value4 = fechas[3]
df.at[4, 'Fecha'] = new_value4

new_value5 = dias_semana[1]
df.at[5, 'Fecha'] = new_value5

new_value6 = fechas[0]
df.at[6, 'Fecha'] = new_value6

new_value7 = dias_semana[0]
df.at[7, 'Fecha'] = new_value7

new_value8 = descripciones[3]
df.at[8, 'Fecha'] = new_value8

new_value9 = descripciones[5]
df.at[9, 'Descripcion'] = new_value9

new_value10 = descripciones[0]
df.at[10, 'Fecha'] = new_value10

new_value11 = fechas[2]
df.at[11, 'Fecha'] = new_value11

new_value12 = dias_semana[2]
df.at[12, 'Fecha'] = new_value12

new_value13 = descripciones[6]
df.at[13, 'Descripcion'] = new_value13

new_value14 = descripciones[4]
df.at[14, 'Descripcion'] = new_value14

# Centrar el encabezado y el texto

estilos = [
    dict(selector="th", props=[("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")])
]

# Aplicar el estilo al DataFrame
df_estilizado = df.style.set_table_styles(estilos)

df

,Nombre,Fecha,Descripcion
0,Talleres de Danzas Peruanas - La Candelaria,None,sabado 20 de enero de 2024
1,TURNSTILE LATIN AMERICA 2024,Desde el lunes 8 de enero del 2024,None
2,Noche Crema 2024,None,"ABONO ROSADO - ANUAL, es un programa que ofrec..."
3,Club Sport Boys Assn. | Trofeo Lizárraga 2024 ...,5 de Marzo del 2024,None
4,Abono Crema Torneo Apertura 2024 - Universitar...,20 de enero del 2024,None
5,Raphael Tour Victoria,Jueves 15 de febrero de 2024 - 20:00,None
6,Abono Rosado Anual 2024,26 de Mayo del 2024,None
7,"Noches de Folklore - Jueves, Viernes y Sábados...",Desde el lunes 8 de enero del 2024,None
8,Tardes de Folklore - Sábados y Domingos en la ...,"Hey Lima, vamos a perrear hasta el suelo con e...",None
9,Temporada de Verano 2024 - Monkeymusic,None,"Jueves, Viernes y Sabados del 2024"


In [4]:
# Teleticket

url = "https://teleticket.com.pe/todos"
response = requests.get(url)
content = response.text
soup = bs.BeautifulSoup(content, 'lxml')

box = soup.find_all('div', {class_ := 'evento--box'})

# Extraemos el nombre de cada concierto, evento o tour

names = soup.find_all('h3')

nombres = [h3.text.strip() for h3 in names]

nombres.pop() # Eliminamos el último vlaor ya que no es un nombre sino se refiere a las 'cookies de la página

# Extraemos el lugar y el tipo del evento

site_class = soup.find_all('p', class_ = 'descripcion')

sitio_clasevento = [p.text.strip() for p in site_class]

# Separamos la lista en dos para una mejor visualización

Locacion = []
Tipo_de_evento = []

for elemento in sitio_clasevento:
    # Dividir el elemento usando "/" como separador
    partes = elemento.split('/')
    
    # Eliminar espacios en blanco alrededor de cada parte
    partes = [parte.strip() for parte in partes]

    # Almacenar las partes en las listas correspondientes
    Locacion.append(partes[0])
    Tipo_de_evento.append(partes[1])

# Extraemos las fechas

dates = soup.find_all('p', class_ = 'fecha')

fechas = [p.text.strip() for p in dates]

# De la lista Locacion extraemos el local, distrito, provincia

parte1 = []
parte2 = []
parte3 = []

for elemento in Locacion:
    # Dividir el elemento usando "-" como separador
    partes = elemento.split('-')
    
    # Eliminar espacios en blanco alrededor de cada parte
    partes = [parte.strip() for parte in partes]

    # Almacenar las partes en las listas correspondientes
    parte1.append(partes[0] if len(partes) > 0 else "")
    parte2.append(partes[1] if len(partes) > 1 else "")
    parte3.append(partes[2] if len(partes) > 2 else "")

lista_combinada = [f'{x}-{y}' for x, y in zip(parte2, parte3)]


data = { 'Nombre':nombres, 'Local':parte1, 'Zona':lista_combinada, 'Tipo de evento':Tipo_de_evento, 'Fecha': fechas}
df = pd.DataFrame(data)
df['Empresa'] = 'Teleticket'

# Centrar el encabezado y el texto

estilos = [
    dict(selector="th", props=[("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")])
]

# Aplicar el estilo al DataFrame
df_estilizado = df.style.set_table_styles(estilos)

df

,Nombre,Local,Zona,Tipo de evento,Fecha,Empresa
0,DIEGO EL CIGALA,Gran Teatro Nacional,SAN BORJA-LIMA,Música,07 de noviembre 2023,Teleticket
1,SARA BARAS,Gran Teatro Nacional,SAN BORJA-LIMA,Música,25 de noviembre 2023,Teleticket
2,PERREO CITY,DISCOTECA COCOS,LINCE-LIMA,Música,09 de diciembre 2023,Teleticket
3,LIT KILLAH EN LIMA,MULTIESPACIO COSTA 21,SAN MIGUEL-LIMA,Entretención,16 de diciembre 2023,Teleticket
4,¡TOUR PARACAS + ICA + CHINCHA 2D/1N CON RUTAS ...,¡TOUR CAÑÓN DE SHUCTO 1N,-,1D CON RUTAS INCAS! - LIMA,15 de enero 2024 - 31 de diciembre 2024,Teleticket
...,...,...,...,...,...,...
182,FARRUKO - LATIN AMERICA TOUR 2023,MULTIESPACIO COSTA 21,SAN MIGUEL-LIMA,Música,22 de junio 2024,Teleticket
183,DREAD MAR I,CONCHA ACÚSTICA DEL CAMPO DE MARTE,JESUS MARIA-LIMA,Música,05 de julio 2024,Teleticket
184,CURSOS LIBRES DE MÚSICA,MONKEYMUSIC,PUEBLO LIBRE-LIMA,Cursos,31 de julio 2024,Teleticket
185,HABANA FÉNIX - CUBA VIBRA,GRAN TEATRO NACIONAL,SAN BORJA-LIMA,Teatro,16 de agosto 2024 - 17 de agosto 2024,Teleticket


# Registro de eventos nacionales

En el siguiente data frame podemos observar el registro de eventos que se realizaron en los periodos del 2018 al 2023, en el cual se mencionan datos importantes como el tipo de evento, la fecha en la que se realizará, el lugar en donde se llevará a cabo, etc.

In [5]:
import pandas as pd
import numpy as np

In [6]:
Eventos_culturales_2018_2023 = pd.read_csv("C:/Users/ferna/OneDrive/Escritorio/TRABAJO_FINAL_LP2/Web-Scraping-de-diversos-eventos-en-el-Per-/DATASET-EVENTOS-CULTURALES-2018-2022 (1).csv", delimiter=';')
Eventos_culturales_2018_2023.head(10) # Para obtener las 10 primeras filas del data frame

,FECHACORTE,TITULO_EVENTO,CATEGORIA,FECHA_INICIO_EVENTO,FECHA_FIN_EVENTO,TIPO_INGRESO,UBICACION,FECHA_PUBLICACION_EVENTO,TIPO_EVENTO,SEDE
0,20230919,Cuentos breves y extraordinarios,Taller y/o curso,20230121,20230121,Ingreso Libre,Hall de Intelectuales,20221227,Presencial,Gran Biblioteca Pública de Lima
1,20230919,Diviértete con los juegos tradicionales,Taller y/o curso,20230113,20230120,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - Rímac
2,20230919,Elaboremos nuestros animales favoritos en Origami,Taller y/o curso,20230119,20230119,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - Breña
3,20230919,Manualidades con material reciclable,Taller y/o curso,20230119,20230126,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - Rímac
4,20230919,Ajedrez para principiantes,Taller y/o curso,20230119,20230126,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - La Victoria
5,20230919,Aprendiendo origami,Taller y/o curso,20230118,20230125,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - Rímac
6,20230919,Canva para principiantes,Taller y/o curso,20230117,20230124,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - La Victoria
7,20230919,Microsoft Word para principiantes,Taller y/o curso,20230113,20230127,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - El Agustino
8,20230919,Juega creando animales con papel,Taller y/o curso,20230112,20230126,Ingreso Libre,Sala,20221227,Presencial,Estación de Biblioteca Pública - El Agustino
9,20230919,Manitos creativas,Taller y/o curso,20230111,20230127,Ingreso Libre,Sala Infantil,20221227,Presencial,Gran Biblioteca Pública de Lima


In [7]:
Eventos_culturales_2018_2023['CATEGORIA'].unique() 

# Observamos que los dintitos tipos de categoria son :

array(['Taller y/o curso', 'Presentación de publicación',
       'Charla, conversatorio y/o conferencia', 'Mediación de lectura',
       'Club de lectura', 'Exposición', 'Narración oral',
       'Cinefórum y/o proyección de cine', 'Otros', 'Teatro para adultos',
       'Teatro para niños', 'Conciertos'], dtype=object)

In [8]:
Eventos_culturales_2018_2023['TIPO_INGRESO'].unique()

# Hay 3 tipos de ingreso:

array(['Ingreso Libre', 'Inscripcion', 'Venta'], dtype=object)

In [9]:
Eventos_culturales_2018_2023['TIPO_EVENTO'].unique()
# Dos tipo de modalidad de los eventos:

array(['Presencial', 'Virtual'], dtype=object)

En el siguiente data frame que veremos a continuación, tenemos solo el registro del primer semestre del 2023. Aqui tambien se nos proporciona información como fecha en la que se realizó el evento, ubicación, categoria,tipo de evento, ya sea presencial o virtual,etc.

In [10]:
Eventos_culturales_2023_I_semestre = pd.read_csv("C:/Users/ferna/OneDrive/Escritorio/TRABAJO_FINAL_LP2/Web-Scraping-de-diversos-eventos-en-el-Per-/DATASET-EVENTOS-CULTURALES-I-SEMESTRE-2023.csv", delimiter=';')
Eventos_culturales_2023_I_semestre

,FECHACORTE,TITULO_EVENTO,CATEGORIA,FECHA_INICIO_EVENTO,FECHA_FIN_EVENTO,TIPO_INGRESO,UBICACION,FECHA_PUBLICACION_EVENTO,TIPO_EVENTO,SEDE
0,20230919,Cazadores de libros,Club de lectura,20230704,20230711,Ingreso Libre,Sala,20230628,Presencial,Estación de Biblioteca Pública - La Victoria
1,20230919,Elaborando mi propio huaco de arcilla,Taller y/o curso,20230721,20230721,Ingreso Libre,Sala Principal,20230628,Presencial,Estación de Biblioteca Pública Tarata-Tacna
2,20230919,Importancia del Qhapaq Ñan: sistema vial andino,Taller y/o curso,20230714,20230714,Ingreso Libre,Sala para Invidentes,20230628,Presencial,Gran Biblioteca Pública de Lima
3,20230919,"Montaje de teatro de títeres del cuento ""Claud...",Taller y/o curso,20230713,20230714,Ingreso Libre,Sala Principal,20230628,Presencial,Estación de Biblioteca Pública Tarata-Tacna
4,20230919,Reusamos y nos recreamos,Taller y/o curso,20230711,20230721,Ingreso Libre,Sala Principal,20230628,Presencial,Estación de Biblioteca Pública Cutervo-Cajamarca
...,...,...,...,...,...,...,...,...,...,...
245,20230919,Aprende matemáticas jugando,Taller y/o curso,20230120,20230127,Ingreso Libre,Sala,20230105,Presencial,Estación de Biblioteca Pública - Comas
246,20230919,Tejiendo mis sueños,Taller y/o curso,20230120,20230127,Ingreso Libre,Sala,20230105,Presencial,Estación de Biblioteca Pública - Comas
247,20230919,Conociendo el sistema Braille,Taller y/o curso,20230119,20230126,Ingreso Libre,Sala,20230105,Presencial,Estación de Biblioteca Pública - El Agustino
248,20230919,Explorando el mundo del origami,Taller y/o curso,20230117,20230124,Ingreso Libre,Sala,20230104,Presencial,Estación de Biblioteca Pública - Comas


Ahora vamos a fusionar ambos data frames para una mejor visualización:

In [11]:
Eventos_culturales_2018_al_2023_I_semestre = pd.concat([Eventos_culturales_2018_2023,Eventos_culturales_2023_I_semestre], ignore_index=True)

Eventos_culturales_2018_al_2023_I_semestre.sample(n=7, random_state=42) #Elegimos 7 filas aleatorias de la data ya fusionada

,FECHACORTE,TITULO_EVENTO,CATEGORIA,FECHA_INICIO_EVENTO,FECHA_FIN_EVENTO,TIPO_INGRESO,UBICACION,FECHA_PUBLICACION_EVENTO,TIPO_EVENTO,SEDE
1413,20230919,Formación parlamentaria,"Charla, conversatorio y/o conferencia",20191006,20191020,Ingreso Libre,Hall principal,20190927,Presencial,Gran Biblioteca Pública de Lima
2168,20230919,Animales cartoneros del Perú,Taller y/o curso,20230623,20230623,Ingreso Libre,Sala,20230525,Presencial,Estación de Biblioteca Pública - Breña
2002,20230919,BIBLIOSESIONES: GESTIÓN CULTURAL EN BIBLIOTECA...,Taller y/o curso,20180808,20180808,Ingreso Libre,Sala de Usos Múltiples (SUM),20180803,Presencial,Biblioteca Nacional del Perú
1996,20230919,BIBLIOSESIONES: LEYENDO CON MI BEBÉ,Taller y/o curso,20180816,20180816,Ingreso Libre,Sala Infantil,20180803,Presencial,Gran Biblioteca Pública de Lima
1103,20230919,Surfeando en la red,Taller y/o curso,20200207,20200228,Ingreso Libre,Sala,20200131,Presencial,Estación de Biblioteca Pública - El Agustino
554,20230919,Álgebra. División de polinomios,Otros,20210408,20210408,Ingreso Libre,Facebook BNP,20210404,Virtual,Biblioteca Nacional del Perú
1660,20230919,TALLER DE LECTURA: LA NOCHE BOCA ARRIBA Y OTRO...,Taller y/o curso,20190504,20190504,Ingreso Libre,Hall de Intelectuales,20190425,Presencial,Gran Biblioteca Pública de Lima
